# Introduction

[Dataset](https://archive-beta.ics.uci.edu/dataset/296/diabetes+130+us+hospitals+for+years+1999+2008) UCI "Diabetes 130-US hospitals for years 1999-2008" [1]

The purpose of this project is to develop a prediction model for <30-day readmissions of diabetic patients. These readmissions can significantly impact Medicare billing, potentially leading to financial penalties for hospitals with high readmission rates. By identifying the key features used by the model to make its classifications, hospitals can develop informed treatment plans to reduce the likelihood of readmissions. Additionally, the model can be used in real-time to predict the probability of a patient"s readmission, providing healthcare professionals with valuable insights for optimizing patient care.


[1] Clore,John, Cios,Krzysztof, DeShazo,Jon & Strack,Beata. (2014). Diabetes 130-US hospitals for years 1999-2008. UCI Machine Learning Repository. https://doi.org/10.24432/C5230J.

In [ ]:
import pandas as pd        # Importing pandas library and renaming it to pd for convenience
import pickle             # Importing the pickle library for reading in saved models later
import numpy as np        # Importing numpy library and renaming it to np for convenience
import matplotlib.pyplot as plt   # Importing matplotlib library and renaming it to plt for convenience
import time               # Importing the time library for tracking the duration of certain operations

pd.options.mode.chained_assignment = None  # Setting pandas chained_assignment option to None to suppress warning messages

seed = 42               # Setting a seed for reproducibility of randomization functions

start_time = time.time()   # Setting a starting time for tracking the duration of certain operations


In [ ]:
plt.style.use("ggplot")   # Setting the default style for Matplotlib to "ggplot"

my_style = {   # Defining a custom style dictionary with new colors and formatting options
    "axes.prop_cycle": plt.cycler(color=["#015bd3", "#fd5305", "#ffd2ea"]),
    "text.color": "#191919",
    "text.color": "#191919",
    "xtick.color": "#191919",
    "ytick.color": "#191919",
    "axes.labelcolor": "#191919",
    "legend.edgecolor": "white",
    "axes.facecolor": "#ebecf0",
}

# Setting the custom style for Matplotlib
plt.style.use(my_style)  


# Data Processing

In [ ]:
df_raw = pd.read_csv("dataset_diabetes/diabetic_data.csv").replace("?", np.nan)
# Reading in the "diabetic_data.csv" file as a pandas DataFrame and storing it in the "df_raw" variable.
# Replacing all "?" values in the DataFrame with NaN (missing) values using the replace() method.

df_raw.head()
# Displaying the first few rows of the DataFrame using the head() method.


In [ ]:
df_raw.info() # Display dataframe info

## Denoise

### Remove columns composed of >30% nan values

Before dropping nan values, I will drop the columns with a signifficant number of nan values.  This will increase the total number of data points after dropping.

In [ ]:
df_raw.weight.isna().value_counts(normalize=True)
# Checking the percentage of missing values in the "weight" column of the DataFrame.
# Using the isna() method to create a Boolean mask for NaN (missing) values in the column.
# Using the value_counts() method to count the number of True/False values in the mask and normalize the counts.

In [ ]:
df = df_raw.copy()   # Creating a copy of the original DataFrame to avoid modifying it directly.

# Creating a list of column names to drop from the DataFrame based on a threshold of missing values.
col_to_drop = list(df_raw.columns[df_raw.isna().sum() > len(df_raw) * 0.3])
col_to_drop.remove("medical_specialty")   # Removing the "medical_specialty" column from the list, as it was deemed important.

list(col_to_drop)   # Displaying the list of column names to be dropped.


### Remove Unused Medications

In [ ]:
unused_meds = ["examide", "citoglipton", "metformin-rosiglitazone"]  # Creating a list of medication columns to be dropped as they only contain None values.

df = df.drop(columns=col_to_drop + unused_meds + ["encounter_id", "patient_nbr"]).dropna()
# Dropping columns that have been deemed unnecessary (col_to_drop and unused_meds),
# as well as the "encounter_id" and "patient_nbr" columns which are patient identifiers and not relevant for the analysis.
# Finally, dropping rows with missing values using the dropna() method.

df.shape   # Displaying the new dimensions (shape) of the DataFrame after the columns and rows have been dropped. 


### Remove patients who died or were transferred to hospice

These patients will not be readmitted and a model is not needed to determine that.

In [ ]:
df = df[~(df.discharge_disposition_id.isin([11, 13, 14, 19, 20, 21, 26]))]

In [ ]:
mappings_raw = pd.read_csv("dataset_diabetes/IDs_mapping.csv", header=None)

admin_type_map = mappings_raw.iloc[0:9, :]
discharge_type_map = mappings_raw.iloc[10:41, :]
admin_source_map = mappings_raw.iloc[42:, :]

dfs = [admin_type_map, discharge_type_map, admin_source_map]

for _df in dfs:
    _df.columns = _df.iloc[0]  # Renaming the columns with the first row values
    _df.drop(_df.index[0], inplace=True)  # Dropping the first row as it is now redundant
    _df.reset_index(drop=True, inplace=True)  # Resetting the index to start at 0 and drop the old index
    _df.set_index(_df.columns[0], inplace=True)  # Setting the index to the first column
    _df.loc[9] = ["Other"]  # Adding an "Other" category to replace IDs not in the mappings
    
admin_type_map  # Displaying the admin_type_map DataFrame

In [ ]:
int_cats = [] # Store the names of these integer categorical columns for later use

for _df in dfs:
    # Get the mapping id values where the description is equivalent to other
    nan_map = _df[_df.description.isin(["Not Mapped", "Not Available", " Not Available", "Unknown/Invalid", np.nan])]
    col_name = _df.index.name
    int_cats.append(col_name)

    nan_id = nan_map.index.values.astype(int)
    
    # Replace the null values with "Other" category
    df[col_name] = df[col_name].replace(nan_id, 9)

# Count the number of values for each category in the "admission_type_id" column
df.admission_type_id.value_counts()


## Feature Engineering

### Create ADM_MERGED feature
This combines the 

In [ ]:
# Combine the values of admission_type_id and medical_specialty columns separated by '_'
df["ADM_MERGED"] = df.admission_type_id.astype(str) + "_" + df.medical_specialty

### Group Infrequent Categories

In [ ]:
# Set threshold for infrequent values
threshold = 0.005 # 0.5%

# Iterate over specified columns and replace infrequent values with 9 for integers and 'other' for objects
for col in ["discharge_disposition_id", "admission_source_id", "medical_specialty", "ADM_MERGED"]:
    # Get infrequent values below the threshold
    infrequent_vals = df[col].value_counts()[df[col].value_counts() < threshold * len(df)].index
    
    # Set replacement value based on column data type
    repl = 9 if df[col].dtype == int else "other"
    
    # Replace infrequent values with replacement value
    df[col] = df[col].replace(infrequent_vals, repl)


### Binary Features

In [ ]:
# Remove rows with unknown or invalid gender and rename gender column to sex
df = df[~(df.gender == "Unknown/Invalid")]
df.rename({"gender":"sex"}, axis=1, inplace=True)

# Convert sex to binary (0 = female, 1 = male)
df.sex = df.sex.apply(lambda x: 0 if x == "Female" else 1)
df.sex.value_counts()

In [ ]:
# Convert "No" values to 0 and "Yes" values to 1 for columns diabetesMed and change
df.diabetesMed = df.diabetesMed.apply(lambda x: 0 if x == "No" else 1)
df.change = df.change.apply(lambda x: 0 if x == "No" else 1)

### Age

In [ ]:
# Make age a continuous variable
df.age = df.age.apply(lambda x: int(np.array(x[1:-1].split("-")).astype(int).mean()))

In [ ]:
# Define function to convert age to age groups
def convert_age_to_group(age):
    if age < 18: # If age is less than 18, return "minor"
        return "minor"
    elif age < 65: # If age is less than 65, return "adult"
        return "adult"
    else: # If age is greater than or equal to 65, return "elderly"
        return "elderly"

# Apply the function to the age column to create a new age_group column
df["age_group"] = df.age.apply(convert_age_to_group)

# Display the value counts of the age_group column
df.age_group.value_counts(normalize=True)


### Diagnoses

https://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/codes

In [ ]:
# Function to categorize ICD9 codes
def get_ICD9_category(value):
        try:
            num = int(float(value)) # convert input to float and then to int
            if (num >=390 and num <=459 ) or num == 785: # Check if circulatory
                category = "circulatory"
            elif (num >= 460 and num <= 519) or num == 786: # Check if respiratory
                category = "respiratory"
            elif (num >= 520 and num <= 579) or num == 787: # Check if digestive
                category = "digestive"
            elif num==250: # Check if diabetes
                category = "diabetes"
            elif num >= 800 and num <= 999: # Check if injury
                category = "injury"
            elif num >= 710 and num <= 739: # Check if musculoskeletal
                category = "musculoskeletal"
            elif (num >= 580 and num <= 629) or num == 788: # Check if genitourinary
                category = "genitourinary"
            elif num >= 140 and num <= 239: # Check if neoplasms
                category = "neoplasms"
            else: # If none of the above, classify as "other"
                category = "other"
        except ValueError: # If the value cannot be converted to float or int, classify as "other"
            category = "other"
        
        return category


In [ ]:
# apply the ICD9 categorization function if it has not already been done
if not df.diag_1.isin(["other"]).any():
    for num in range(1,4):
        col_name = f"diag_{num}"
        df[col_name] = df[col_name].apply(get_ICD9_category)

        # print the column name and its value counts
        print(col_name)
        print(df[col_name].value_counts(normalize=True), "\n")


In [ ]:
# Subset the dataframe to just the medication columns
drugs = df.iloc[:, 19:39].copy()

# Identify the columns that do not have exactly 4 unique values
drugs_to_drop = drugs.columns[drugs.nunique() != 4]

# Drop the columns from the original dataframe
df.drop(columns=drugs_to_drop, inplace=True)


In [ ]:
# Create a copy of the drugs columns in the dataframe
num_changes = drugs.copy()

# Iterate through each column in num_changes
for col in num_changes:
    
    # Apply a lambda function to convert medication values to 0 if "No" or "Steady", 1 otherwise
    num_changes[col] = num_changes[col].apply(lambda x: 0 if x in ["No", "Steady"] else 1)
    
# Sum the values in each row of num_changes and create a new column in df called "num_med_changes"
df["num_med_changes"] = num_changes.sum(axis=1)


In [ ]:
# Review changes before proceeding
df.info()

## Adressing Class Imbalance via Under Sampling

In [ ]:
# View normalized values to observe class imbalance
df.readmitted.value_counts(normalize=True)

In [ ]:
# Extract target variable, convert to binary classification
y = df.readmitted
y = y.apply(lambda x: 1 if x == "<30" else 0)

print(len(y)) # print number of samples
fig, ax = plt.subplots(figsize=(7, 5))
to_plot = y.value_counts() # get the count of each class
bars = to_plot.plot(kind="bar", ax=ax) # plot the counts as a bar chart
for i, v in enumerate(to_plot):
    ax.annotate(str(round(v / len(y) * 100)) + "%", xy=(i, v), ha="center", va="bottom") # add percentage labels to the bars

ax.set_xticklabels([">30D or No Readmin", "<30D Readmin"], rotation=0) # label the x-axis

fig.savefig("Presentation/Images/48657_before_undersamp.png", dpi=300) # save the figure as an image


In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Drop the target variable from X
X = df.drop(columns=["readmitted"])

# Perform random undersampling to balance the classes
X, y = RandomUnderSampler(random_state=seed, sampling_strategy=1.0).fit_resample(X, y)

# Print the new length of y (the target variable)
print(len(y))

# Create a bar chart to visualize the class balance after undersampling
fig, ax = plt.subplots(figsize=(7, 5))
to_plot = y.value_counts()
bars = to_plot.plot(kind="bar", ax=ax)
for i, v in enumerate(to_plot):
    ax.annotate(str(round(v / len(y) * 100)) + "%", xy=(i, v), ha="center", va="bottom")
ax.set_xticklabels([">30D or No Readmin", "<30D Readmin"], rotation=0)

# Save the figure to a file
fig.savefig("Presentation/Images/10816_after_undersamp.png", dpi=300)

In [ ]:
# Select all categorical columns (object) and the previously identified integer categorical columns (int_cats)
cat_cols = X.select_dtypes(include=object).columns.to_list() + int_cats

# Select all continuous columns by dropping the categorical columns from the dataframe
cont_cols = X.drop(columns=cat_cols).columns.tolist()

# Loop through each categorical column and ensure that it is represented as a string
for column in cat_cols:
    X.loc[:, column] = X[column].astype(str)

# Create dummy variables for each categorical column and store it in a new dataframe called dummies
dummies = pd.get_dummies(X[cat_cols])

# Drop all categorical columns from the original dataframe
X.drop(columns=cat_cols, inplace=True)

# Join the dummies dataframe with the original dataframe
X = X.join(dummies)


In [ ]:
# Import necessary modules
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=seed, stratify=y)

# Instantiate the MinMaxScaler
scaler = MinMaxScaler()

# Scale the continuous columns in the training set
X_train[cont_cols] = scaler.fit_transform(X_train[cont_cols])

# Scale the continuous columns in the testing set
X_test[cont_cols] = scaler.transform(X_test[cont_cols])

# Print the shapes of the training and testing sets
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
# Import necessary libraries for model evaluation
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Define a function to print classification metrics and confusion matrix
def eval_model(model):
    for label, y_true, _X in (("Train", y_train, X_train), ("Test", y_test, X_test)):
        y_pred = model.predict(_X)

        fig, axes = plt.subplots(ncols=2, figsize=(10,4))
        
        # Display the classification report, which includes precision, recall, f1-score, and support
        metrics = classification_report(y_true, y_pred)
        ax=axes[1]
        ax.axis("off")
        ax.text(0.5, 0.25, metrics, size=12, ha="center", transform=ax.transAxes)

        ax = axes[0]
        # Create the confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        # Create confusion matrix plot, which shows the counts of true positives, false positives, true negatives, and false negatives
        sns.heatmap(cm, annot=True, fmt="d", cmap="plasma", ax=ax)
        ax.set_ylabel("True label")
        ax.set_xlabel("Predicted label")

        fig.tight_layout()

# Models

In [ ]:
# Import necessary libraries for computing class weights
from sklearn.utils import compute_class_weight

# Compute class weights
class_weights = [0.95, 1.0]
class_weights_dict = dict(enumerate(class_weights))


## Random Forest

In [ ]:
# Import the necessary library for RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Create a RandomForestClassifier object with a fixed random_state for reproducibility
rand_forest = RandomForestClassifier(random_state=seed)

# Train the model on the training data
rand_forest.fit(X_train, y_train)

# Evaluate the model using the previously defined eval_model() function
eval_model(rand_forest)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import clone

# Clone the model and perform a grid search
rand_forest_gscv = clone(rand_forest)

# param_grid = {
#     "n_estimators": [200, 500, 750],
#     "max_depth": [None, 5, 7],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 4],
#     "max_features": ["sqrt", "log2"],
#     "bootstrap": [True, False],
#     "class_weight" : ["balanced", class_weights_dict]
# }


# grid_search = GridSearchCV(rand_forest_gscv, param_grid, scoring="roc_auc", n_jobs=-1)
# grid_search.fit(X_train, y_train)

# grid_search.best_params_

# eval_model(grid_search)

In [ ]:
params = {
    "bootstrap": True, # Whether or not to use bootstrapping to resample the dataset
    "class_weight": {0: 0.9, 1: 1.0}, # Weight given to each class label in the loss function
    "max_depth": 7, # Maximum depth of the decision trees in the forest
    "max_features": "sqrt", # Maximum number of features to consider when looking for the best split
    "min_samples_leaf": 1, # Minimum number of samples required in a leaf node
    "min_samples_split": 2, # Minimum number of samples required to split an internal node
    "n_estimators": 750 # Number of trees in the forest
}

# Set the hyperparameters of the random forest model
rand_forest_gscv.set_params(**params)

# Train the random forest model using the training data
rand_forest_gscv.fit(X_train, y_train)

# Evaluate the model performance on the train and test data using the previously defined function
eval_model(rand_forest_gscv)

## SVC

In [ ]:
# Import necessary libraries for building a support vector machine model
from sklearn.svm import SVC

# Instantiate the support vector machine model with a specified random state
svc = SVC(random_state=42)

# Fit the support vector machine model to the training data
svc.fit(X_train, y_train)

# Evaluate the performance of the support vector machine model on the training and testing sets
eval_model(svc)


In [ ]:
# Create a clone of the support vector machine model
svc_gscv = clone(svc)

# Define the parameters to be used in the grid search
params = {
    "kernel" : "rbf",  # Kernel type used in the algorithm
    "gamma" : "scale", # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'
    "class_weight": {0:0.9, 1:1.0} # Set weight of the classes, the first value is for class 0 and the second for class 1
}

# Set the parameters for the support vector machine model
svc_gscv.set_params(**params)

# Train the support vector machine model with the grid search optimized parameters
svc_gscv.fit(X_train, y_train)

# Evaluate the performance of the support vector machine model using the evaluation function
eval_model(svc_gscv)


## LightGBM

In [ ]:
# Import necessary libraries for building a LightGBM model
from lightgbm import LGBMClassifier

# Instantiate a LightGBM classifier model with a random state for reproducibility
lgbm = LGBMClassifier(random_state=seed)

# Train the model on the training data
lgbm.fit(X_train, y_train)

# Evaluate the model using the defined evaluation function
eval_model(lgbm)


In [ ]:
lgbm_gscv = clone(lgbm)

# param_grid = {
#     "learning_rate": [0.01, 0.1, 0.05],
#     "n_estimators": [100, 500],
#     "max_depth": [3, 5, None],
#     "num_leaves": [5, 10, 20],
#     "min_child_samples": [10, 20, 30],
#     "reg_lambda": [0.0, 0.1, 0.5],
#     "colsample_bytree": [0.5, 0.7, 0.9],
#     "subsample": [0.5, 0.7, 0.9],
#     "class_weight" : ["balanced", class_weights_dict]
# }

# grid_search = GridSearchCV(lgbm_gscv, param_grid, scoring="roc_auc", n_jobs=-1)
# grid_search.fit(X_train, y_train)

# grid_search.best_params_

# eval_model(grid_search)

In [ ]:
params = {
    "class_weight": {0: 0.9, 1: 1.0}, # Class weights to address imbalance
    "colsample_bytree": 0.5, # fraction of columns to use at each split
    "learning_rate": 0.05, # shrinkage rate for gradient descent
    "max_depth": None, # maximum depth of tree (None = no limit)
    "min_child_samples": 30, # minimum number of samples in a child (leaf) node
    "n_estimators": 100, # number of trees to use
    "num_leaves": 10, # maximum number of leaves in a tree
    "reg_lambda": 0.0, # regularization parameter for L2 regularization
    "subsample": 0.5 # fraction of rows to use at each split
}

# Set the model parameters to the best parameter combination found through grid search (uncomment this line to use)
# params = grid_search.best_params_

lgbm_gscv.set_params(**params) # set the parameters of the model
lgbm_gscv.fit(X_train, y_train) # fit the model
eval_model(lgbm_gscv) # evaluate the model using the defined evaluation function


## CatBoost

In [ ]:
from catboost import CatBoostClassifier

# Initialize the CatBoostClassifier model
catboost = CatBoostClassifier(random_state=seed, verbose=0)

# Fit the model on the training data
catboost.fit(X_train, y_train)

# Evaluate the model on both the training and testing data using the `eval_model` function
eval_model(catboost)


In [ ]:
catboost_gscv = clone(catboost)

# param_grid = {
#     "learning_rate": [0.1, 0.05, 0.01],
#     "n_estimators": [100, 500, 1000],
#     "depth": [3, 5, 7],
#     "l2_leaf_reg": [0.0, 0.1, 0.5],
#     "colsample_bylevel": [0.5, 0.7, 0.9],
#     "subsample": [0.5, 0.7, 0.9],
#     "class_weights" : [class_weights]
# }
# # "roc_auc"
# grid_search = GridSearchCV(catboost_gscv, param_grid, scoring="roc_auc", n_jobs=-1)
# grid_search.fit(X_train, y_train)

# grid_search.best_params_

# eval_model(grid_search)

In [ ]:
params = {
    "class_weights": [0.94, 1.0],  # weights for each class, with a higher weight assigned to the minority class
    "colsample_bylevel": 0.5,  # fraction of columns to subsample at each split level
    "depth": 5,  # depth of the tree
    "l2_leaf_reg": 0.1,  # L2 regularization coefficient for leaf weights
    "learning_rate": 0.01,  # learning rate for gradient descent
    "n_estimators": 1000,  # number of trees to build
    "subsample": 0.7  # fraction of rows to subsample at each split level
}

# Set the hyperparameters for the CatBoost model using GridSearchCV
catboost_gscv.set_params(**params)

# Fit the CatBoost model with the training data
catboost_gscv.fit(X_train, y_train)

# Evaluate the CatBoost model with the eval_model function
eval_model(catboost_gscv)

## Deep Neural Network

In [ ]:
def analyze_model(_history, _model, _test_data=X_test, y_true=y_test):
    fig, axes = plt.subplots(2, 2, figsize=(10,8))

    # Generate predictions on the test data
    y_pred = _model.predict(_test_data)

    if y_pred.shape[1] == 1:
        y_pred = list((y_pred > 0.5).astype(int))
    else:
        y_pred = np.argmax(y_pred, axis=1)
        y_true = np.argmax(y_true, axis=1)

    # Print model metrics report
    metrics = classification_report(y_true, y_pred)
    ax=axes[1][1]
    ax.axis("off")
    ax.text(0.5, 0.25, metrics, size=12, ha="center", transform=ax.transAxes)

    # Generate a confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Plot the confusion matrix
    ax=axes[1][0]
    sns.heatmap(cm, annot=True, fmt="d", cmap="plasma", ax=ax)
    ax.set_ylabel("True label")
    ax.set_xlabel("Predicted label")

    # Plot the training and validation accuracy
    ax = axes[0][0]

    ax.plot(_history.history["accuracy"])
    ax.plot(_history.history["val_accuracy"])
    ax.set_title("Model Accuracy")
    ax.set_ylabel("Accuracy")
    ax.set_xlabel("Epoch")
    ax.legend(["Train", "Val"])

    # Plot the training and validation loss
    ax = axes[0][1]
    ax.plot(_history.history["loss"])
    ax.plot(_history.history["val_loss"])
    ax.set_title("Model Loss")
    ax.set_ylabel("Loss")
    ax.set_xlabel("Epoch")
    ax.legend(["Train", "Val"])

    fig.tight_layout()

In [ ]:
# from keras import Sequential
# from keras.layers import Dense, Dropout, BatchNormalization, Dropout, Bidirectional
# from keras.callbacks import EarlyStopping
# from keras.optimizers import Adam
# from keras.regularizers import l1,l2

# # Define a sequential model using Keras
# model = Sequential([
#     Dense(64, "relu", input_dim=X_train.shape[1]), # Input layer with 64 neurons and "relu" activation function
#     BatchNormalization(), # Batch normalization layer to normalize activations
#     Dropout(0.2), # Dropout layer to prevent overfitting by randomly dropping out 20% of the neurons
#     Dense(32, "relu", kernel_regularizer=l2(0.01)), # Hidden layer with 32 neurons and "relu" activation function with L2 regularization (0.01)
#     Dropout(0.2), # Another dropout layer to prevent overfitting
#     Dense(32, "relu", kernel_regularizer=l2(0.01)), # Another hidden layer with 32 neurons and "relu" activation function with L2 regularization (0.01)
#     Dense(16, "relu"), # Another hidden layer with 16 neurons and "relu" activation function
#     Dense(1, "sigmoid") # Output layer with 1 neuron and "sigmoid" activation function (binary classification)
# ])

# # Compile the model with Adam optimizer (with a learning rate of 1e-4), binary_crossentropy loss function, and accuracy and recall as evaluation metrics
# model.compile(Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy", "Recall"])

# # Train the model with training and validation sets, with batch size 32 and 50 epochs, and add EarlyStopping callback to stop training if validation loss doesn't improve for 6 consecutive epochs
# history = model.fit(
#     X_train.values, y_train.values, 
#     batch_size=32, 
#     epochs=50, 
#     validation_data=(X_test.values, y_test.values),
#     callbacks=[
#         EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)],
#     class_weight={0:.93, 1:1.0} # Set class weights to address class imbalance in the dataset
# )

# # Save the trained model and history to files for future use
# model.save("RNN_info/RNN.h5")
# with open("RNN_info/RNN_history.pickle", "wb") as file:
#     pickle.dump(history, file)

# # Analyze and visualize the training history of the model
# analyze_model(history, model)


In [ ]:
from keras.models import load_model

# Load the saved model and history
model = load_model("RNN_info/RNN.h5")
with open("RNN_info/RNN_history.pickle", "rb") as file:
    history = pickle.load(file)

# Evaluate the model and display metrics
analyze_model(history, model)

# Visualizations

## Model's Perfromance

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Define a list of models to evaluate
models = [rand_forest_gscv, svc_gscv, lgbm_gscv, catboost_gscv, model]

# Define a list of model names for display purposes
model_names = ["Random Forest", "SVC", "LightGBM", "CatBoost", "Recurrent Neural Net"]

# Create empty lists to store the evaluation metrics for each model
accs, target_f1s, avg_f1s = [], [], []

# Loop through each model to evaluate its performance
for _model in models:
    # Make predictions on the test data using the current model
    preds = _model.predict(X_test)
    preds = list((preds > 0.5).astype(int))

    # Calculate the precision, recall, and F1-score for each class (0 and 1)
    precision, recall, f1_score, support  = precision_recall_fscore_support(y_test, preds,)
    
    # Calculate the accuracy score
    acc = accuracy_score(y_test, preds)
    
    # Extract the F1-score for the positive class (class 1)
    target_f1 = f1_score[1]
    
    # Calculate the average F1-score across both classes
    avg_f1 = f1_score.mean()

    # Append the evaluation metrics for the current model to their respective lists
    accs.append(round(acc*100))
    target_f1s.append(round(target_f1*100))
    avg_f1s.append(round(avg_f1*100))

# Define a list of the evaluation metrics to display in the bar chart
metrics = ["Accuracy", "Target F1-Score", "Average F1-Score"]

# Create variables to set the position and width of each bar in the bar chart
x_pos = np.arange(len(model_names))
width = 0.2

# Create a bar chart to display the evaluation metrics for each model
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(x_pos - width, accs, width, label="Accuracy")
ax.bar(x_pos + width, target_f1s, width, label="Target F1-Score")
ax.bar(x_pos, avg_f1s, width, label="Average F1-Score")

# Set the x-tick labels to the model names and display them vertically
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, rotation=90)

# Set the y-label to "Score"
ax.set_ylabel("Score")

# Add a legend to the chart
ax.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

# Annotate each bar with its respective value
for i, v in enumerate(accs):
    ax.annotate(str(v) + "%", xy=(i-width, v), ha="center", va="bottom")
for i, v in enumerate(target_f1s):
    ax.annotate(str(v) + "%", xy=(i+width, v), ha="center", va="bottom")
for i, v in enumerate(avg_f1s):
    ax.annotate(str(v) + "%", xy=(i, v), ha="center", va="bottom")

# Adjust the layout of the figure to prevent overlapping text
fig.tight_layout()

# Save the figure to a file
fig.savefig("Presentation/Images/ModelComparison.png", dpi=300)


## Feature Importance

In [ ]:
# Define replacements for string preprocessing of labels
replacements = (
    ("discharge_disposition_id_3", "Discharged to SNF"),
    ("discharge_disposition_id_1", "Discharged to Home"),
    ("discharge_disposition_id_22", "Discharged to Rehab"),
    ("num_", "number_"), # Rename columns starting with "num_" to "number_"
    ("ADM_MERGED_2", "Urgent Care to"), # Replace ADM_MERGED_2 with "Urgent Care to"
    ("diabetesMed", "On Diabetes Meds") # Replace diabetesMed with "On Diabetes Meds"
)

# Function to preprocess labels
def prep_label(string):
    for to, repl in replacements:
        string = string.replace(to, repl)
    
    return string.replace("_", " ").title()


# Calculate feature importances and select the top 11 features
importances = pd.Series(rand_forest_gscv.feature_importances_, X_train.columns)
_X = pd.concat([X_train, X_test])
_y = pd.concat([y_train, y_test])
important_feats = importances.sort_values(ascending=True)[-11:]

# Plot the feature importances
fig, ax = plt.subplots(figsize=(10,6))
ax.vlines([0], len(important_feats), -1, colors="#fd5305") # Add vertical line to show the importance cutoff
ax.tick_params(axis='y', which='major', labelsize=12) # Adjust y-axis tick label size
(important_feats * _X.corrwith(y)[important_feats.index].apply(lambda x: -1 if x < 0 else 1)).plot(kind="barh", ax=ax) # Plot the feature importances
labels = [prep_label(label) for label in important_feats.index] # Apply label preprocessing
ax.set_yticklabels(labels) # Set y-tick labels
fig.tight_layout()

fig.savefig("Presentation/Images/lgbm_importances.png", dpi=300) # Save the plot as a PNG file


## Partial Dependance of Discharge Facilities

In [ ]:
_X = pd.concat([X_train, X_test]).median()
discharge_ids = _X.index[_X.index.str.contains("discharge_disposition_id")]
_X.loc[discharge_ids] = 0

preds = pd.Series(dtype=float)

for id in discharge_ids:
    x = _X.copy()
    x.loc[id] = 1
    x = np.reshape(x.values, (1, -1))

    pred = lgbm_gscv.predict_proba(x)

    id = id.split("_")[-1]
    name = discharge_type_map.loc[id].description
    preds.loc[name] = pred[0][1]

In [ ]:
preds.sort_values(inplace=True)
preds.index = ["Home", "Home with Health Service", "ICF", "Left AMA", "Inpatient at Same Hospital", "Different Short-Term Hospital", "SNF", "Inpatient at Different Hospital", "Rehab or In-House Rehab"]

fig, ax = plt.subplots(figsize=(10, 10))
preds.plot(kind="barh", ax=ax)
ax.set_xlabel("predicted probability of readmission".title())
ax.tick_params(axis='y', which='major', labelsize=14)
# ax.set_yticklabels(ax.get_yticklabels(), rotation=30)
fig.tight_layout()
fig.savefig("Presentation/Images/DischargeTypes.png", dpi=300)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

fig, ax = plt.subplots(figsize=(10,13))

PartialDependenceDisplay.from_estimator(rand_forest_gscv, X, important_feats.index, ax=ax, kind="both")
fig.tight_layout()

In [ ]:
import os

def alert(title="title", text="text"):
    os.system("""
              osascript -e 'display alert "{}" message "{}"'
              """.format(title, text))

# Alert me when notebook is finished so that I can multitask
total_time = (time.time() - start_time) / 60 
    
alert("Notebook Completed!", f"Completed in {total_time} minutes.")